In [1]:
import numpy as np
from scipy.interpolate import interp1d
import sys
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from matplotlib.patches import Rectangle

In [2]:
%gui qt

In [3]:
%matplotlib inline

In [4]:
from Algo2.generate_smooth_curve import generate_smooth_curve
from Algo2.diff_algorithm import calculate_diff_between_input_and_curve, calculate_jumps
from utilities.export_files import excel_format

In [8]:
inputFile = 'testin.txt'
smoothedOutput = "output.xls"
thresholdOutput = 0.75
window = 0.0375
threshold = 0.0025 #initial threshold guess

### load data

In [6]:
#isolate x and y data
data = np.loadtxt(inputFile)
x = data[:,0]
y = data[:,1]

In [7]:
x2y2 = generate_smooth_curve(x, y, window)
x2 = x2y2['x2']
y2 = x2y2['y2']

excel_format(x2, y2, smoothedOutput)

In [ ]:
# generate cubic spline interpolation and plot
poly = interp1d(x2, y2, kind='cubic')
xi = np.linspace(x2[0], x2[x2.size - 1], num=(x2[x2.size - 1] - x2[0])/.001, endpoint=True)

# calculate diff between input and curve
ydiff = calculate_diff_between_input_and_curve(x2)

In [ ]:
# plot with threshold values marked
fig, ax = plt.subplots(2)
ori, = ax[1].plot(x, y, 'o-', color='g') # plot original points
cs, = ax[1].plot(xi, poly(xi), color='c') # plot cubic spline
ax[1].plot(x2, y2, 'o', color = 'c') # plot smoothed curve
plt.xlabel("Wavelength (Angs)")
plt.ylabel("(Raw - OB) / OB")
plt.title("Neutron Imaging Graph")
ws = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
plt.legend([ws, ori, cs], ("window size: " + str(window * 2), "original data", "cubic spline"), loc='best')
maxjumps = calcjumps(threshold)
for i in range(0, maxjumps.size): # mark x-values of points exceeding threshold
    ax[1].axvline(x=maxjumps[i], color='r')
plt.subplots_adjust(bottom = 0.2)
ax[0].set_position([0.12, 0.05, 0.70, 0.03])
ax[1].set_position([0.12, 0.2, 0.70, 0.70])
sthresh = Slider(ax[0], 'Threshold', 0.000, 0.005, valinit = threshold, valfmt='%1.5f')

# update x-values when threshold slider changes
def update(val):
	threshold = sthresh.val
	maxjumps = calculate_jumps(threshold, ydiff, x2)
	fig.canvas.draw_idle()
	while len(ax[1].lines) > 3:
		ax[1].lines[-1].remove()
	for i in range(0, maxjumps.size):
		ax[1].axvline(x=maxjumps[i], color='r')
sthresh.on_changed(update)

plt.show()